In [ ]:
from src.data_processor import ecommerceProcessor
from src.visualization_engine import ecommerceViz
from src.segmentation import customerSegmentation

In [ ]:
processor = ecommerceProcessor()
processor.run_queries(test_=True)
processor.prep_events()
processor.prep_session()
processor.prep_agg_conversion()

# Conversion Rates Across Key Events

Key Observations:
- A really small fraction (~5%) of customers convert from viewing the page to adding items to cart
- The most dramatic drop in conversion happens from beginning checkout to completing a purchase (over 50% reduction in the conversion rate). This is extremely high, and I suggest examining the checkout process. Possible causes:
    - There may be too many steps in the process and customers get frustrated and give up
    - There may be a bug with required information, etc. that make users second guess the purchase
    - There could be nothing wrong with the interface, but unexpected charges are being applied (i.e. shipping, cannot use code, cannot apply a sale) that is turning customers away

In [ ]:
visualizer = ecommerceViz(processor)
visualizer.plot_conversion_rate()

In [ ]:
visualizer.plot_conversion_rate(remove_pageview=True)

# Conversion Rates over Time

Observations:
- Traffic was growing throughout November and December, likely ahead of the Christmas holiday in the US and other countries
- A large spike came mid-December. Was there a sales campaign at that time? 
    - While total purchaes seemed to increase, it appears there was a drop in conversion rates
    - It's possible these new visitors were net-new to the site, but may have struggled with navigation or the checkout process (or were only enticed by the sales offerings, not necessarily interest in the products)
    - This warrants further exploration with the sales and marketing team along with product
- There is very clear weekly seasonality, with Sundays appearing to be the strongest sales day

In [ ]:
visualizer.plot_conversion_rates_over_time(remove_pageview=True)

# Trends in Aggregate Events

There is an anomaly in the first few weeks of the collection period where there are fewer adds to cart than purchases. This is impossible and needs to be investigated.

In [ ]:
visualizer.plot_conversion_events_over_time()

In [ ]:
visualizer.plot_conversion_events_over_time(remove_pageview=True)

# Segmentation

In [ ]:
segmenter = customerSegmentation(processor)

segmenter.prep_segment_data()
segmenter.prep_clustering_data()
segmenter.create_kmeans(centers=5)
segmenter.add_customer_segments()

In [ ]:
visualizer.plot_conversion_rate(plot_segments=True)